In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/NYSU/BabyCrying_Cutter/

/content/drive/MyDrive/NYSU/BabyCrying_Cutter


In [3]:
!pip install pytube
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 KB 3.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pytube
import subprocess
import os
import numpy as np
from pydub import AudioSegment

In [5]:
text_file_dir = "./video_urls.txt"
save_dir = "BabyCryingWav/"
output_dir = "Data/"

In [6]:
with open(text_file_dir, "r") as f:
    links = f.read().splitlines()

for i, link in enumerate(links):
  try:
    yt = pytube.YouTube(link)
    stream = yt.streams.filter(only_audio=True).first()
    temp_file_path = stream.download(output_path=save_dir, filename="temp.mp4")
        
    output_file_path = os.path.join(save_dir, f"{i}.wav")

    command = f"ffmpeg -i {temp_file_path} -acodec pcm_s16le -ac 1 -ar 16000 BabyCryingWav/{i}.wav"
    !{command}

    os.remove(temp_file_path)
  except Exception as e:
    print(f"Error processing {link}: {e}")

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [7]:
slice_duration = 4000
baby_cry_threshold = -30

for index, filename in enumerate(os.listdir(save_dir)):
  if filename.endswith(".wav"):
    try:
      audio = AudioSegment.from_wav(os.path.join(save_dir, filename))

      samples = audio.get_array_of_samples()
      samples_hp = np.array(samples)
      samples_hp = samples_hp.astype(np.float64)
      samples_hp -= np.mean(samples_hp)
      samples_hp /= 32767.0
      fc = 1000
      b = fc / (audio.frame_rate / 2)
      a = [1, -1]
      samples_hp = np.convolve(samples_hp, b, mode='same')
      samples_hp = np.convolve(samples_hp, a, mode='same')
      samples_hp *= 32767.0

      baby_crying_indices = np.where(samples_hp <= baby_cry_threshold)[0]
      baby_crying_ranges = []
      if len(baby_crying_indices) > 0:
        start = baby_crying_indices[0]
        for i in range(1, len(baby_crying_indices)):
          if baby_crying_indices[i] - baby_crying_indices[i - 1] > slice_duration:
            end = baby_crying_indices[i - 1]
            baby_crying_ranges.append(samples[start:end])
            start = baby_crying_indices[i]
        end = baby_crying_indices[-1]
        baby_crying_ranges.append(samples[start:end])


      output_dir_path = os.path.join(output_dir, str(index))  
      os.makedirs(output_dir_path, exist_ok=True)

      for i, baby_crying_chunk in enumerate(baby_crying_ranges):
        output_file_name = str(i) + ".wav"
        baby_crying_audio = AudioSegment(
          data=baby_crying_chunk.tobytes(),
          sample_width=audio.sample_width,
          frame_rate=audio.frame_rate,
          channels=audio.channels
        )
        baby_crying_audio = baby_crying_audio.set_frame_rate(22050).set_channels(1)
        baby_crying_audio = baby_crying_audio.set_sample_width(2)

        if len(baby_crying_audio) >= slice_duration:
          baby_crying_audio = baby_crying_audio[:slice_duration]
        else:
          padding_duration = slice_duration - len(baby_crying_audio)
          silence = AudioSegment.silent(duration=padding_duration)
          baby_crying_audio = baby_crying_audio + silence

        output_file_name = f"{i}.wav"
        output_file_path = os.path.join(output_dir_path, output_file_name)
        baby_crying_audio.export(output_file_path, format="wav")
        print(output_file_path)
    except Exception as e:
      print(f"Error processing {filename}: {e}")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Data/3/907.wav
Data/3/908.wav
Data/3/909.wav
Data/3/910.wav
Data/3/911.wav
Data/3/912.wav
Data/3/913.wav
Data/3/914.wav
Data/3/915.wav
Data/3/916.wav
Data/3/917.wav
Data/3/918.wav
Data/3/919.wav
Data/3/920.wav
Data/3/921.wav
Data/3/922.wav
Data/3/923.wav
Data/3/924.wav
Data/3/925.wav
Data/3/926.wav
Data/3/927.wav
Data/3/928.wav
Data/3/929.wav
Data/3/930.wav
Data/3/931.wav
Data/3/932.wav
Data/3/933.wav
Data/3/934.wav
Data/3/935.wav
Data/3/936.wav
Data/3/937.wav
Data/3/938.wav
Data/3/939.wav
Data/3/940.wav
Data/3/941.wav
Data/3/942.wav
Data/3/943.wav
Data/3/944.wav
Data/3/945.wav
Data/3/946.wav
Data/3/947.wav
Data/3/948.wav
Data/3/949.wav
Data/3/950.wav
Data/3/951.wav
Data/3/952.wav
Data/3/953.wav
Data/3/954.wav
Data/3/955.wav
Data/3/956.wav
Data/3/957.wav
Data/3/958.wav
Data/3/959.wav
Data/3/960.wav
Data/3/961.wav
Data/3/962.wav
Data/3/963.wav
Data/3/964.wav
Data/3/965.wav
Data/3/966.wav
Data/3/967.wav
Data/3/968.wav
Data/3/969.wav
Data/3/970.wav
Data